<a href="https://colab.research.google.com/github/auzaluis/upsa_mod_202402/blob/main/personalidad/01-script-ETL-personalidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Tema 01: Carga de datos**

### **Importando base de datos**

In [ ]:
# Google Auth
from google.colab import auth
auth.authenticate_user()

In [ ]:
# API Client
from google.auth import default
creds, _ = default()

In [ ]:
# gspread authorization
import gspread
gc = gspread.authorize(creds)

In [ ]:
# Accediendo al Google Sheet
url_personalidad = "https://docs.google.com/spreadsheets/d/1IQ_RxxTSmBKHTExlxboIRNlMov_F6RyqdcOPrflCv_w/edit?usp=sharing"
gsheets = gc.open_by_url(url_personalidad)
sheets = gsheets.worksheet("Respuestas de formulario 1").get_all_values()

In [ ]:
type(sheets)

In [ ]:
# Convirtiendo la lista en df
import pandas as pd
df = pd.DataFrame(sheets[1:], columns=sheets[0])

### **Inspección data frame**

In [ ]:
type(df)

In [ ]:
# Ver cantidad de filas y columnas
df.shape

In [ ]:
# Ver primeras filas
df.head()

In [ ]:
# Ver info general y nombre de columas
df.info()

In [ ]:
# Tipo de datos
df['Sexo'].dtype

In [ ]:
# Número de filas
len(df)

In [ ]:
# Número de columnas
len(df.columns)

### **Tema 02: Transformación de datos**

### **Valores perdidos**

#### **Identificación de los NAs**

In [ ]:
# Los NAs están strings vacíos
import numpy as np
df.replace("", np.nan, inplace=True)

In [ ]:
df['Escribe tu edad exacta'].isna().value_counts()

#### **Tratando de los NAs**

In [ ]:
df["Escribe tu edad exacta"].dtype

In [ ]:
df["Escribe tu edad exacta"] = pd.to_numeric(df["Escribe tu edad exacta"], errors="coerce")

In [ ]:
df["Escribe tu edad exacta"].dtype

In [ ]:
df["Escribe tu edad exacta"].mean()

In [ ]:
edad_promedio = df["Escribe tu edad exacta"].mean()

##### **Reemplazo por la media**

In [ ]:
# Creando df2
df2 = df.copy()

In [ ]:
# Reemplazo por la media
df2["edad2"] = df2["Escribe tu edad exacta"].fillna(edad_promedio)

In [ ]:
df2[["Escribe tu edad exacta", "edad2"]]

**Reubicando edad2**

In [ ]:
# Crear una lista con los nombres de las columnas
lista_columnas = list(df2.columns)

In [ ]:
# Quitando temporalmente edad2 de la lista
lista_columnas.remove("edad2")

In [ ]:
# Obteniendo el índice (posición) de la columna "after"
lista_columnas.index("Escribe tu edad exacta")

In [ ]:
# Insertando "edad2" después de "Escribe tu edad exacta"
lista_columnas.insert(lista_columnas.index("Escribe tu edad exacta") + 1, "edad2")

In [ ]:
# Reordenando y guardando en df2
df2 = df2[lista_columnas]

In [ ]:
df2.head()

**Creando relocate() en python**

In [ ]:
def relocate(df, columna, after):
  lista_columnas = list(df.columns)
  lista_columnas.remove(columna)
  lista_columnas.insert(lista_columnas.index(after) + 1, columna)
  return df[lista_columnas]

In [ ]:
relocate(
    df = df2,
    columna = "edad2",
    after = "Escribe tu edad exacta"
)

##### **Eliminar la fila completa**

In [ ]:
df2.shape

In [ ]:
df2 = df2.dropna()

In [ ]:
df2.shape

### **Estandarización de variables**

#### **Normalización**

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# instanciando StandardScaler()
normalizador = StandardScaler()

In [ ]:
# normalizando
normalizador.fit_transform(df2[["edad2"]])

In [ ]:
# creando df3
df3 = df2.copy()

In [ ]:
df3["edadZ"] = normalizador.fit_transform(df3[["edad2"]])

In [ ]:
# reubicando edadZ despues de edad2
df3 = relocate(
    df = df3,
    columna = "edadZ",
    after = "edad2"
)

In [ ]:
df3[["edad2", "edadZ"]]

#### **Rango**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# instanciando MinMaxScaler()
rango = MinMaxScaler()

In [ ]:
# convirtiendo a rango
df3["edadRango"] = rango.fit_transform(df3[["edad2"]])

In [ ]:
df3 = relocate(
    df = df3,
    columna = "edadRango",
    after = "edadZ"
)

In [ ]:
df3[["edad2", "edadRango"]]

### **Agrupaciones**

#### **Numéricas**

In [ ]:
cortes = [-float('inf'), 18, 21, float('inf')]
etiquetas = ["18 o menos", "19 a 21", "Más de 21"]

In [ ]:
df3['edadGR'] = pd.cut(
    x = df3['edad2'],
    bins = cortes,
    labels = etiquetas
)

In [ ]:
df3 = relocate(
    df = df3,
    columna = "edadGR",
    after = "edad2"
)

In [ ]:
# frecuencias de edadGR
df3['edadGR'].value_counts()

#### **Categóricas**

In [ ]:
# Índices
df3.info()

In [ ]:
# Valores únicos
pd.unique(df3.iloc[:,8])

In [ ]:
# Frecuencias
df3.iloc[:,8].value_counts()

In [ ]:
df3.iloc[:,8].isin(['Totalmente verdadero', 'Un poco verdadero'])

Dummitización

In [ ]:
# Función condicional normal
def top2box(x):
  if x == "Un poco verdadero" or x == "Totalmente verdadero":
    return 1
  else:
    return 0

In [ ]:
# aplicando la función
df3.iloc[:,8].apply(top2box).value_counts()

In [ ]:
# función lambda
df3.iloc[:,8].apply(lambda x: 1 if x == "Un poco verdadero" or x == "Totalmente verdadero" else 0)

In [ ]:
# frecuencias
df3.iloc[:,8].apply(lambda x: 1 if x == "Un poco verdadero" or x == "Totalmente verdadero" else 0).value_counts()

In [ ]:
df3.iloc[:,8].apply(lambda x: 1 if x in ["Un poco verdadero", "Totalmente verdadero"] else 0).value_counts()

#### **Ordinales**

In [ ]:
import numpy as np

In [ ]:
# niveles
pd.unique(df3.iloc[:,8])

In [ ]:
# Cuando las condiciones son muchas (5 en este caso), es mejor usar select()
condiciones = [
    df3.iloc[:,8] == "Totalmente falso",
    df3.iloc[:,8] == "Un poco falso",
    df3.iloc[:,8] == "No lo sé",
    df3.iloc[:,8] == "Un poco verdadero",
    df3.iloc[:,8] == "Totalmente verdadero"
]

# Hacer coincidir el orden
opciones = [1,2,3,4,5]

In [ ]:
# usando select
ordinal = np.select(
    condlist = condiciones,
    choicelist = opciones,
    default = np.nan
)

In [ ]:
ordinal

In [ ]:
# no confundir return_counts de numpy con value_counts de pandas
np.unique(ordinal, return_counts = True)

In [ ]:
# para aplicar value_counts() primero se debe convertir a pd.Series
pd.Series(ordinal).value_counts()

#### **Bucles**

Forma convencional / standard

In [ ]:
# creando df4
df4 = df3.copy()

In [ ]:
df4.columns

In [ ]:
# crear una lista vacía
frases = []

In [ ]:
# se ejecuta un blucle para llenar la lista
for col in df4.columns:
  if col.startswith("Según tu"):
    frases.append(col)

In [ ]:
frases

In [ ]:
# aplicando el bucle para dummitizar, recordemos que creamos la función top2box
for frase in frases:
  df4[frase] = df4[frase].apply(top2box)

In [ ]:
df4[frases]

In [ ]:
df4.iloc[:,8].value_counts()

Forma Pythonica

In [ ]:
# creando df5
df5 = df3.copy()

In [ ]:
# lista vacía
frases = []

In [ ]:
# usando comprehension list
frases = [col for col in df5.columns if col.startswith("Según tu")]

In [ ]:
frases

In [ ]:
# aplicando el bucle para dummitizar, esta vez con lambda, como si la función top2box no existiera
for frase in frases:
  df5[frase] = df5[frase].apply(lambda x: 1 if x in ["Un poco verdadero", "Totalmente verdadero"] else 0)

In [ ]:
df5[frases]

In [ ]:
df5[frases] == df4[frases]

### **Tema 03: Manipulación de datos**

#### **Selección de columnas**

In [ ]:
df5['Sexo']

In [ ]:
df5[['Sexo']]

In [ ]:
# dobles corchetes obligatorios para seleccionar más de una col
df5[['Sexo', 'Escribe tu edad exacta']]

In [ ]:
# eliminar una columna
df5.drop(columns='Marca temporal')

In [ ]:
# seleccionar las cols que comienzan con...
df5[[col for col in df5.columns if col.startswith("¿Cuánto")]]

In [ ]:
# seleccionar las cols que terminan con...
df5[[col for col in df5.columns if col.endswith("]")]]

In [ ]:
# contains
df5.filter(like='edad', axis=1)

#### **Selección de filas**

In [ ]:
# Seleccionar filas cuando Sexo = 'Mujer'
df5[df5['Sexo'] == 'Mujer'].head(3)

In [ ]:
# Omitir filas cuando Sexo no es igual "Hombre"
df5[df5['Sexo'] != 'Hombre'].head(3)

In [ ]:
# Seleccionar filas cuando edad2 es mayor o igual a 20
df5[df5['edad2'] >= 20].head(3)

In [ ]:
# Seleccinar filas cuando edad2 está entre 18 y 21
df5[(df5['edad2'] >= 18) & (df5['edad2'] <= 21)].head(3)

In [ ]:
# Seleccinar filas cuando edad2 está entre 18 y 21
# Seleccionar filas cuando Sexo = 'Mujer'
df5[
    (df5['edad2'] >= 18) &
    (df5['edad2'] <= 21) &
    (df5['Sexo'] == 'Mujer')
].head(3)

#### **Renombrando columnas**

In [ ]:
df6 = df5.copy()

**APPs**

In [ ]:
df6.columns

**Paso 1:** Creamos una lista con los nuevos nombres

In [ ]:
apps = ["TikTok", "Instagram", "Facebook", "YouTube"]

**Paso 2:** Crear un dict que contenga los nombres antiguos como keys y los nuevos como values

In [ ]:
# zip() crea tuplas (nombre antiguo, nombre nuevo)
# dict() almacena la tupla en un diccionario
apps_dict = dict(zip(
 [col for col in df6.columns if col.startswith("¿Cuánto")],
 apps
))

**Paso 3:** Renombrando

In [ ]:
df6.rename(apps_dict, axis=1, inplace=True)

In [ ]:
df6.info()

**Frases**

**Paso 0:** Convertir la lista a Pandas Series

In [123]:
frases2 = pd.Series(frases)

**Paso 1:** Crear una lista con los nuevos nombres

In [124]:
frases2 = frases2.str.extract('\\[(.+)\\]')[0]

In [ ]:
frases2

**Paso 2:** Crear un dict que contenga los nombres antiguos como keys y los nuevos como values

In [127]:
frases_dict = dict(zip(frases, frases2))

In [ ]:
frases_dict

**Paso 3:** Renombrando

In [129]:
df6.rename(frases_dict, axis=1, inplace=True)

In [ ]:
df6.info()

#### **Pivotado**

Pivot Longer

In [133]:
apps

['TikTok', 'Instagram', 'Facebook', 'YouTube']

In [134]:
df7 = df6.melt(
    # identificadores
    id_vars = ['Marca temporal', 'Sexo', 'edad2'],
    value_vars = apps,
    var_name = 'app',
    value_name = 'time'
)

In [135]:
df7.head()

,Marca temporal,Sexo,edad2,app,time
0,9/08/2021 12:23:46,Hombre,16.0,TikTok,0:00:00
1,9/08/2021 12:25:01,Mujer,16.0,TikTok,0:50:00
2,9/08/2021 12:26:31,Hombre,17.0,TikTok,8:00:00
3,9/08/2021 12:28:49,Mujer,22.0,TikTok,22:35:00
4,9/08/2021 12:31:43,Hombre,16.0,TikTok,7:00:00


Pivot Wider

In [136]:
df8 = df7.pivot(
    index = ['Marca temporal', 'Sexo', 'edad2'],
    columns = 'app',
    values = 'time'
)

In [ ]:
df8.head()

In [ ]:
df8.info()

In [139]:
# recuperar las cols que están como index
df8.reset_index(inplace=True)

In [ ]:
df8.info()

In [ ]:
df8.head()

### **Tema 04: Detección de outliers**

In [147]:
df7.head()

,Marca temporal,Sexo,edad2,app,time
0,9/08/2021 12:23:46,Hombre,16.0,TikTok,0:00:00
1,9/08/2021 12:25:01,Mujer,16.0,TikTok,0:50:00
2,9/08/2021 12:26:31,Hombre,17.0,TikTok,8:00:00
3,9/08/2021 12:28:49,Mujer,22.0,TikTok,22:35:00
4,9/08/2021 12:31:43,Hombre,16.0,TikTok,7:00:00


In [149]:
df7['time'].dtype

dtype('O')

Transformar horas a número

In [150]:
# split() separa texto
'22:35:00'.split(':')

['22', '35', '00']

In [151]:
# función conversora
def tiempo_a_num(x):
  x = x.split(':')
  return int(x[0]) + int(x[1])/60 + int(x[2])/3600

In [153]:
tiempo_a_num('22:30:00')

22.5

In [155]:
df7['time'] = df7['time'].apply(tiempo_a_num)

In [158]:
df7.head()

,Marca temporal,Sexo,edad2,app,time
0,9/08/2021 12:23:46,Hombre,16.0,TikTok,0.000000
1,9/08/2021 12:25:01,Mujer,16.0,TikTok,0.833333
2,9/08/2021 12:26:31,Hombre,17.0,TikTok,8.000000
3,9/08/2021 12:28:49,Mujer,22.0,TikTok,22.583333
4,9/08/2021 12:31:43,Hombre,16.0,TikTok,7.000000


#### **Boxplots**

In [157]:
import plotly.express as px

In [160]:
boxplot_01 = px.box(df7, y = 'time')

In [161]:
boxplot_01.show()

Aperturar por app

In [164]:
px.box(df7, x = 'app', y = 'time')